<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#custom-&quot;assert&quot;-function" data-toc-modified-id="custom-&quot;assert&quot;-function-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>custom "assert" function</a></span></li><li><span><a href="#EWMA-Calculations" data-toc-modified-id="EWMA-Calculations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EWMA Calculations</a></span><ul class="toc-item"><li><span><a href="#Show-our-working" data-toc-modified-id="Show-our-working-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Show our working</a></span></li></ul></li><li><span><a href="#Variance-and-Volatility" data-toc-modified-id="Variance-and-Volatility-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Variance and Volatility</a></span><ul class="toc-item"><li><span><a href="#Using-MercurPy" data-toc-modified-id="Using-MercurPy-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Using MercurPy</a></span></li></ul></li><li><span><a href="#As-a-tool" data-toc-modified-id="As-a-tool-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>As a tool</a></span></li><li><span><a href="#EWMA-Correlation" data-toc-modified-id="EWMA-Correlation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>EWMA Correlation</a></span><ul class="toc-item"><li><span><a href="#Show-our-working" data-toc-modified-id="Show-our-working-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Show our working</a></span></li><li><span><a href="#Single-function" data-toc-modified-id="Single-function-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Single function</a></span></li><li><span><a href="#Over-pairs-of-series-without-a-matching-lifespan" data-toc-modified-id="Over-pairs-of-series-without-a-matching-lifespan-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Over pairs of series without a matching lifespan</a></span></li><li><span><a href="#As-a-function-of-pairs-of-Series" data-toc-modified-id="As-a-function-of-pairs-of-Series-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>As a function of pairs of Series</a></span></li><li><span><a href="#from-MercurPy" data-toc-modified-id="from-MercurPy-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>from MercurPy</a></span></li></ul></li><li><span><a href="#Spread-Level,-Dispersion-and-Vol" data-toc-modified-id="Spread-Level,-Dispersion-and-Vol-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Spread Level, Dispersion and Vol</a></span></li><li><span><a href="#Spread-correlation" data-toc-modified-id="Spread-correlation-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Spread correlation</a></span></li></ul></div>

# EWMA calibrations revised


## custom "assert" function

We will compare "realised" results calculated here with "expected" results. We will highlight differences when this is outside a given level of tolerance.

In [1]:
import math

def my_assert(expected, realised, bp=15):
    if math.isclose(expected, realised,rel_tol = 0, abs_tol=10 ** -bp):
        return realised
    else:
        return "returned {0}, diff vs expected of {1}".format(realised, expected-realised)
    


In [2]:
my_assert(4,2+2)

4

In [3]:
my_assert(4,2+1.9999999,6)

3.9999998999999997

In [4]:
my_assert(4,2+1.9999999,7)

'returned 3.9999998999999997, diff vs expected of 1.0000000028043132e-07'

[Link to the workbook containing the results](V:\MACS_Projects\Assumptions_Mercury_Implementation\EWMA_Formula\EWMA_Testing_MercuryReplication_and_Handover.xlsm)


## EWMA Calculations

### Show our working

Given a *series* of data $x$ that exists over all points from $ 1\to t $;

The general form of an *EWMA* average of the series,  $ \bar{x}_t $ of is:

$$ \bar{x}_t = \lambda*\bar{x}_{t-1}+(1-\lambda)x_t $$

Where an initalisation value, $\bar{x}_0$ is given.

For instance, the *series* we will use in Target Setting is a **Time Series** of data - where the $t$ values are *evenly-spaced* date values.  

In [5]:
import pandas as pd
import mercurpy as mp
import numpy as np

url1 = "http://sdi/Api/models/2019-03-31/TimeSeries.MarketData.Equity.ExcessReturn.3m/Best_Views/RW/NONE/NONE/E_GBP/NONE/?newFormat=True"
url2 = "http://sdi/Api/models/2019-03-31/TimeSeries.MarketData.Equity.ExcessReturn.3m/Best_Views/RW/NONE/NONE/E_USD/NONE/?newFormat=True"

df = mp.model_dataframe_from_url_list([url1,url2],"_valueDateUtc","EquityAsset")
df = df.reset_index()
df["_valueDateUtc"] = pd.to_datetime(df["_valueDateUtc"])
df.set_index(["_valueDateUtc"], inplace = True)
df.sort_index(inplace=True)

In [6]:
returns = df['E_USD'].apply(pd.to_numeric, errors='ignore').to_frame().dropna()
returns

,E_USD
_valueDateUtc,
1960-03-31 00:00:00+00:00,-0.108461
1960-06-30 00:00:00+00:00,0.020698
1960-09-30 00:00:00+00:00,-0.074707
1960-12-31 00:00:00+00:00,0.071239
1961-03-31 00:00:00+00:00,0.106261
1961-06-30 00:00:00+00:00,-0.010592
1961-09-30 00:00:00+00:00,0.024628
1961-12-31 00:00:00+00:00,0.072056
1962-03-31 00:00:00+00:00,-0.036114


In [7]:
lambda_value = 0.99
variance_initial_value = 0.011957245526038
vintage_date = pd.Timestamp(2018, 6, 30)

In [8]:
# recursive function
def ewma(series,lambda_value, initial_value=None):
    """ base recursive function for EWMA, where the input series is a pandas Series or DataFrame
    We use "Squeeze" to allow a dataframe to be used 
        - this squeezes any DF down to one column, or leaves an existing series as it is.
    Lambda is input directly.
    The initial value is optional. If not supplied, the mean of the input series is used
    """ 
    
    series = series.squeeze()
    
    if initial_value == None:
        initial_value = np.mean(series)
        
    T = [initial_value]
    for i in range (0,len(series)):
        #print("value {0}: prev value  = {1} current return value  = {2}".format(i,series[i],R[i]))
        T.append(lambda_value * T[i] + (1 - lambda_value) * series[i])       
    return pd.Series(T[1:], index=series.index)

In [9]:
# Check that we return the expected values from the test spreadsheet.
# Spreadsheet Var @vintage date = 0.0126430573181749
my_assert(0.0126430573181749, ewma(returns, lambda_value,initial_value=0).loc[vintage_date].item())

0.012643057318174892

## Variance and Volatility

We can expand on the general EWMA function to calculate the *Variance* of returns.  

The general form of an *EWMA* Variance of the series,  $ \sigma^2_t $ of is:

$$ \sigma^2_t = \lambda*\sigma^2_{t-1}+(1-\lambda){(x_t-\bar{x}_t)}^2 $$

Note that this is equivalent to the general EWMA function above, where we replace $ x_t $ with ${(x_t-\bar{x}_t)}^2 $.  

It will be useful to have a separate function to calculate the **Mean Distance**  ${(x_t-\bar{x}_t)} $ within the Variance function.  



In [10]:
# Having a seprarate "ewma_r_bar_df" function will be handy when calculating covars later
def ewma_mean_dist(series,lambda_value, x_bar=None, mean_initial_value=None):
    """ Calculate the distance/deviation from a mean value.        
        The mean value can be passed in as a constant. If not provided, the EWMA mean is used as a rolling series.        
    """
    returns_df = series.copy(deep=True)
    if x_bar == None:
        # rolling ewma
        returns_series = returns_df.squeeze()        
        ewma_means = ewma(returns_df, lambda_value, mean_initial_value).squeeze()              
        r_bar_df = (returns_series - ewma_means)               
    else:        
        r_bar_df = (returns_df - x_bar)
    
    return r_bar_df         


The main difference between our past implementations of EWMA has been the choice of $ \bar{x}_t $.  

Previously, for our *Equity Volatility* implementation we have assumed that $ \bar{x}_t = 0 $ across all points in time. 

We can replicate this here by passing a constant value for the `x_bar` argument.

In [11]:
ewma_mean_dist(returns, lambda_value, x_bar=0, mean_initial_value=0).loc[vintage_date].item()

0.02910553295238256

Going forward, we want to use the **EWMA Mean** as $ \bar{x}_t $. We can do this by leaving the `x_bar` argument undefined, and providing the `mean_initial_value` for the EMWA Mean series.

In [12]:
ewma_mean_dist(returns, lambda_value, mean_initial_value=0).loc[vintage_date].item()

0.01646247563420767

We can now use the `ewma_mean_dist` and `ewma` functions to create a Variance function. We will then illustrate how this can be used to return results for both the older and new desired approaches.  

In [13]:
def ewma_var(series,lambda_value, variance_initial_value, x_bar=None, mean_initial_value=None):
    """ Variance version of the EWMA function.
        Here, the series that is measured is the square of observations minus a mean value.
        The mean value can be passed in as a constant. If not provided, the EWMA mean is used as a rolling series.        
    """          
    r_bar_df = ewma_mean_dist(series,lambda_value=lambda_value, x_bar=x_bar, mean_initial_value=mean_initial_value)      
        
    ewma_r_bar_df = ewma(r_bar_df ** 2,lambda_value, variance_initial_value)
    
    return ewma_r_bar_df

In [14]:
# Previous approach with x_bar = 0
# Spreadsheet Var @vintage date = 0.00684018192232017
my_assert(0.00684018192232017, ewma_var(returns, lambda_value, variance_initial_value= 0.011957245526038, x_bar=0).loc[vintage_date].item())

0.00684018192232016

In [15]:
# New approach with EWMA mean, intialised at = 0
# Spreadsheet Var @vintage date = 0.00684018192232017
my_assert(0.00662491493444609, ewma_var(returns, lambda_value, variance_initial_value= 0.011957245526038, mean_initial_value=0).loc[vintage_date].item())

0.006624914934446092

We can calculate the *EWMA Volatilty* of the Series by taking the square root of the variance. Furthermore, we will calculate the *Annualised EWMA Volatilty* by multiplying this value by the square root of the *steps per annum*.  

In [16]:
def ewma_vol(series,lambda_value, variance_initial_value, steps_per_annum = 4, x_bar=None, mean_initial_value=None):
    """ turn a variance into an annualised volatility, by providing the number of steps per year
        Will default to quarterly i.e. steps_per_annum = 4
    """
    var = ewma_var(series,lambda_value, variance_initial_value=variance_initial_value,x_bar=x_bar,mean_initial_value=mean_initial_value)
    vol = var.apply(lambda v: (steps_per_annum ** 0.5) * v ** 0.5)
    return vol

In [17]:
# EWMA vol with mean = 0 (like for equity assets in previous implementation)
my_assert(0.165410784682501, ewma_vol(returns, 0.99, variance_initial_value= 0.011957245526038, x_bar=0).loc[vintage_date].item())

0.1654107846825008

In [18]:
# EWMA vol with rolling EWMA mean (initial value not given, so the average of the initial series is used)
my_assert(0.162787160850555, ewma_vol(returns, lambda_value, variance_initial_value= 0.011957245526038, mean_initial_value=0).loc[vintage_date].item())

0.1627871608505547

### Using MercurPy

We have made the functions used above avaliable in the **MercurPy** Python add-in, so that they can be called in later spec documents. 

This will be converted separately into the Mercury (C#) code base and regression tested against the original spec

In [19]:
import mercurpy as mp

In [20]:
my_assert(0.0126430573181749, mp.ewma(returns, lambda_value,initial_value=0).loc[vintage_date].item())

0.012643057318174892

In [21]:
my_assert(0.00662491493444609, mp.ewma_var(returns, lambda_value, variance_initial_value=0.011957245526038, mean_initial_value = 0).loc[vintage_date].item())

0.006624914934446092

In [22]:
my_assert(0.162787160850555, mp.ewma_vol(returns, lambda_value, variance_initial_value=0.011957245526038, mean_initial_value=0).loc[vintage_date].item())

0.1627871608505547

Repeat this analysis for E_GBP

In [23]:
gbp_XS_returns = df['E_GBP'].apply(pd.to_numeric, errors='ignore').to_frame().dropna()

In [24]:
my_assert(0.00864134861332517, mp.ewma(gbp_XS_returns, lambda_value,initial_value=0).loc[vintage_date].item())

0.008641348613325165

In [25]:
my_assert(0.00656601451923979, mp.ewma_var(gbp_XS_returns, lambda_value, 0.006960204394972, mean_initial_value = 0).loc[vintage_date].item())

0.0065660145192397955

In [26]:
my_assert(0.162061895820576, mp.ewma_vol(gbp_XS_returns, lambda_value, 0.006960204394972, mean_initial_value=0).loc[vintage_date].item())

0.16206189582057587

## As a tool

We will create a tool with the following TMT setup.

![](images/ewma_TMT.png)

The core of the tool will require access to the *ewma*, *ewma_mean_dist*, *ewma_var* and *ewma_vol* functions. The settings will determine which function will be required to be run over the input TimeSeries to produce the output EWMA TimeSeries.     

Together, we will need a meta-set of parameters that covers all functions. These will be the settings of our TMT tool.

We will also have an optional "Vintage Date" model. This may be used as the *EndDate* setting, if it is provided.  

The outputs are both optional. Both may be present in config.  

For the *Structured_Output*, we will construct a model to push containing a single parameter, that we will name through the *structured_model_parameter_name* setting. Its value will be the last value entry in the output EWMA TimeSeries. We can also include the date of the last entry as another parameter, if the *structured_model_date_parameter_name* is set.  

The *TimeSeries_Output* will be a TimeSeries model containing the output EWMA TimeSeries.

The full Settings will be defined as follows:

In [27]:
# settings for the preferred EWMA Volatility case using a rolling EWMA mean as the "x_bar" value and no given mean inital value
settings_ewma = {
    "lambda_value": lambda_value,
    "variance_initial_value": variance_initial_value, # will trigger the use of ewma_vol/ewma_var function if not provided, then it is the ewma (average) that we return
    "x_bar": None,
    "mean_initial_value": 0, 
    "volatility_steps_per_annum": 4, # will trigger the use of ewma_vol function, which will return an annualised Vol
    "EndDate": "VINTAGE_DATE", # filter out the input series beyond this: can be VINTAGE_DATE, CALIBRATION_DATE or None (take the last value)
    "structured_model_parameter_name": "UncVol.Value", # pluck the final entry in the time series,and create a parameter in the structured model with this name
    "structured_model_date_parameter_name": "VintageDate" # if included, then add the date used as a parameter in the structured model with this name
} 

In [28]:
settings_ewma['lambda_value']

0.99

The core of the tool will operate in a similar way to the function below.

In [29]:
def ewma_tool(series, settings):
    
    if settings_ewma['variance_initial_value'] is None:
        return ewma(series, 
                    lambda_value=settings_ewma['lambda_value'],
                    initial_value=settings_ewma['mean_initial_value'])
    else:
        if settings_ewma['volatility_steps_per_annum'] is None:
            return ewma_var(series,
                            lambda_value=settings_ewma['lambda_value'],
                            variance_initial_value=settings_ewma['variance_initial_value'],
                            x_bar=settings_ewma['x_bar'],
                            mean_initial_value=settings_ewma['mean_initial_value'])
        else:
            return ewma_vol(series,
                            lambda_value=settings_ewma['lambda_value'],
                            variance_initial_value=settings_ewma['variance_initial_value'],
                            steps_per_annum=settings_ewma['volatility_steps_per_annum'],
                            x_bar=settings_ewma['x_bar'],
                            mean_initial_value=settings_ewma['mean_initial_value'])

# testing this against the expected USD result @vintage_date
my_assert(0.162787160850555, ewma_tool(returns, settings_ewma).loc[vintage_date].item())

0.1627871608505547

Except here we calculate EWMA first, then pluck or filter the data. The TMT tool should filter the input series first.  

It will then arrange the output models as defined in the TMT for that run.

## EWMA Correlation

This is currently implemented in the *Unconditional Correlation* tool. Here, we will show how to update this calculation to allow for EWMA means. 

This should give the same answer as in the existing tool when a constant mean equal to the *mean of each individual series, over the correlation period* is used. Which will be harder to set up and replicate, so we will need to consider if that case will be needed when this correction is applied in a TMT tool. 

### Show our working

Note that we are truncating the data here by a vintage date. As we take the *mean* of this series over a period of time, this will change as the vintage date changes.  

The correlation, $\rho_t(x,y)$ between a pair of TimeSeries $(x,y)$ can be defined as:

$$ \rho_t(x,y) = \frac{\sigma_t^2(x,y)}{\sigma_t^2(x)\sigma_t^2(y)} $$

Where the *covariance* $ \sigma_t^2(x,y) $ is defined by the EWMA function:

$$ \sigma_t^2(x,y) = \lambda\sigma_{t-1}^2(x,y)+(1-\lambda)(x_t-\bar{x_t})(y_t-\bar{y_t}) $$

We will have the same concerns over which mean value to use in this calculation.  

We will first work through the "old" approach, where we assume that: 

+ Both series have a common initialisation value
+ We calculate Variance using a **Mean over the lifespan**, i.e. the average
+ We calculate means over the *common lifespan* of the pair of series

In [30]:
# take pair from our downloaded data
df = df.apply(pd.to_numeric, errors='ignore')
pair = pd.concat([df['E_USD'],df['E_GBP']], axis=1)
pair = pair[pair.index <= pd.Timestamp(2018, 6, 30).tz_localize('GMT')]
pair.columns = ["Left_Returns", "Right_Returns"]
pair

,Left_Returns,Right_Returns
_valueDateUtc,,
1960-03-31 00:00:00+00:00,-0.108461,-0.053257
1960-06-30 00:00:00+00:00,0.020698,-0.030606
1960-09-30 00:00:00+00:00,-0.074707,0.040560
1960-12-31 00:00:00+00:00,0.071239,-0.040048
1961-03-31 00:00:00+00:00,0.106261,0.126850
1961-06-30 00:00:00+00:00,-0.010592,-0.043942
1961-09-30 00:00:00+00:00,0.024628,-0.088271
1961-12-31 00:00:00+00:00,0.072056,0.022813
1962-03-31 00:00:00+00:00,-0.036114,-0.029016


Note that this pair of series have the same length. For both the existing method in Mercury, and the EWMA version we will define here, we want to only consider data over *the shared lifespan of both series*.  

In Pandas, `.dropna()` on a pair of well-formed TimeSeries will give us the data set that we need.

In [31]:
pair = pair.dropna()

In [32]:
np.mean(pair)

Left_Returns     0.009279
Right_Returns    0.007645
dtype: float64

In [33]:
# constant Mu version
returns_minusMu = pair.apply(lambda C: ewma_mean_dist(C, lambda_value, x_bar=np.mean(C)))
returns_minusMu

,Left_Returns,Right_Returns
_valueDateUtc,,
1960-03-31 00:00:00+00:00,-0.117741,-0.060902
1960-06-30 00:00:00+00:00,0.011419,-0.038251
1960-09-30 00:00:00+00:00,-0.083986,0.032915
1960-12-31 00:00:00+00:00,0.061960,-0.047693
1961-03-31 00:00:00+00:00,0.096981,0.119204
1961-06-30 00:00:00+00:00,-0.019871,-0.051588
1961-09-30 00:00:00+00:00,0.015349,-0.095917
1961-12-31 00:00:00+00:00,0.062777,0.015167
1962-03-31 00:00:00+00:00,-0.045393,-0.036661


In [34]:
returns_minusMu['Covar'] = returns_minusMu.iloc[:,0] * returns_minusMu.iloc[:,1]
returns_minusMu

,Left_Returns,Right_Returns,Covar
_valueDateUtc,,,
1960-03-31 00:00:00+00:00,-0.117741,-0.060902,0.007171
1960-06-30 00:00:00+00:00,0.011419,-0.038251,-0.000437
1960-09-30 00:00:00+00:00,-0.083986,0.032915,-0.002764
1960-12-31 00:00:00+00:00,0.061960,-0.047693,-0.002955
1961-03-31 00:00:00+00:00,0.096981,0.119204,0.011561
1961-06-30 00:00:00+00:00,-0.019871,-0.051588,0.001025
1961-09-30 00:00:00+00:00,0.015349,-0.095917,-0.001472
1961-12-31 00:00:00+00:00,0.062777,0.015167,0.000952
1962-03-31 00:00:00+00:00,-0.045393,-0.036661,0.001664


In [35]:
returns_minusMu['EWMA_Covar'] = ewma(returns_minusMu['Covar'].to_frame(),lambda_value, initial_value=(0.005625 * 0.5))
returns_minusMu['Left_EWMA_Var'] = ewma_var(returns_minusMu['Left_Returns'].to_frame(), lambda_value, variance_initial_value = 0.005625, x_bar=0)
returns_minusMu['Right_EWMA_Var'] = ewma_var(returns_minusMu['Right_Returns'].to_frame(), lambda_value, variance_initial_value = 0.005625, x_bar=0)
returns_minusMu

,Left_Returns,Right_Returns,Covar,EWMA_Covar,Left_EWMA_Var,Right_EWMA_Var
_valueDateUtc,,,,,,
1960-03-31 00:00:00+00:00,-0.117741,-0.060902,0.007171,0.002856,0.005707,0.005606
1960-06-30 00:00:00+00:00,0.011419,-0.038251,-0.000437,0.002823,0.005652,0.005564
1960-09-30 00:00:00+00:00,-0.083986,0.032915,-0.002764,0.002767,0.005666,0.005520
1960-12-31 00:00:00+00:00,0.061960,-0.047693,-0.002955,0.002710,0.005647,0.005487
1961-03-31 00:00:00+00:00,0.096981,0.119204,0.011561,0.002799,0.005685,0.005574
1961-06-30 00:00:00+00:00,-0.019871,-0.051588,0.001025,0.002781,0.005632,0.005545
1961-09-30 00:00:00+00:00,0.015349,-0.095917,-0.001472,0.002738,0.005578,0.005582
1961-12-31 00:00:00+00:00,0.062777,0.015167,0.000952,0.002720,0.005562,0.005528
1962-03-31 00:00:00+00:00,-0.045393,-0.036661,0.001664,0.002710,0.005527,0.005486


In [36]:
my_assert(0.00470872965966373, returns_minusMu['EWMA_Covar'].loc[pd.Timestamp(2018,6,30)])

0.004708729659663728

In [37]:
returns_minusMu['EWMA_Correl'] = returns_minusMu['EWMA_Covar'] / (returns_minusMu['Left_EWMA_Var'] * returns_minusMu['Right_EWMA_Var']) ** 0.5
returns_minusMu

,Left_Returns,Right_Returns,Covar,EWMA_Covar,Left_EWMA_Var,Right_EWMA_Var,EWMA_Correl
_valueDateUtc,,,,,,,
1960-03-31 00:00:00+00:00,-0.117741,-0.060902,0.007171,0.002856,0.005707,0.005606,0.504931
1960-06-30 00:00:00+00:00,0.011419,-0.038251,-0.000437,0.002823,0.005652,0.005564,0.503430
1960-09-30 00:00:00+00:00,-0.083986,0.032915,-0.002764,0.002767,0.005666,0.005520,0.494851
1960-12-31 00:00:00+00:00,0.061960,-0.047693,-0.002955,0.002710,0.005647,0.005487,0.486835
1961-03-31 00:00:00+00:00,0.096981,0.119204,0.011561,0.002799,0.005685,0.005574,0.497134
1961-06-30 00:00:00+00:00,-0.019871,-0.051588,0.001025,0.002781,0.005632,0.005545,0.497600
1961-09-30 00:00:00+00:00,0.015349,-0.095917,-0.001472,0.002738,0.005578,0.005582,0.490739
1961-12-31 00:00:00+00:00,0.062777,0.015167,0.000952,0.002720,0.005562,0.005528,0.490613
1962-03-31 00:00:00+00:00,-0.045393,-0.036661,0.001664,0.002710,0.005527,0.005486,0.492119


In [38]:
my_assert(0.74933278113088, returns_minusMu['EWMA_Correl'].loc[pd.Timestamp(2018,6,30)])

0.7493327811308803

### Single function

We can summarise the steps above to create a single function that will calculate EWMA correlation.  

We will also introduce here our new/preferred approach. Here, we assume that: 

+ Both series have a **separate** initialisation value
+ We calculate Variance using an **EWMA Mean**
+ We calculate means over the *common lifespan* of the pair of series

The `use_ewma_mean` argument decides on what mean calculation will be used in the variance.  

When this is True, `left_var_init` and `right_var_init` should be provided. If not, then we use the common `var_init` value.  

In [39]:
# take pair
df = df.apply(pd.to_numeric, errors='ignore')
pair_in = pd.concat([df['E_USD'],df['E_GBP']], axis=1)
pair_in = pair_in[pair_in.index <= pd.Timestamp(2018, 6, 30).tz_localize('GMT')]
pair_in

,E_USD,E_GBP
_valueDateUtc,,
1960-03-31 00:00:00+00:00,-0.108461,-0.053257
1960-06-30 00:00:00+00:00,0.020698,-0.030606
1960-09-30 00:00:00+00:00,-0.074707,0.040560
1960-12-31 00:00:00+00:00,0.071239,-0.040048
1961-03-31 00:00:00+00:00,0.106261,0.126850
1961-06-30 00:00:00+00:00,-0.010592,-0.043942
1961-09-30 00:00:00+00:00,0.024628,-0.088271
1961-12-31 00:00:00+00:00,0.072056,0.022813
1962-03-31 00:00:00+00:00,-0.036114,-0.029016


In [40]:
def ewma_correl_df(pair_df, lambda_value, covar_init, var_init=None, left_var_init=None, right_var_init=None, use_ewma_mean=True):
    """ From a DataFrame containing a pair of values, calculate the EWMA correlation between them.
        To ensure that we work over the shared lifespan of both series, we drop missing values which will remove any
        extra trailing data from the longer series. 
    """   
    
    df = pair_df.copy()
    df.columns = ["Left_Returns", "Right_Returns"]
    df = df.dropna()
    
    # Ensure that we work over the shared lifespan of both series.
    
    if var_init != None: # If inital var is set, it will override the left/right specific values
        left_var_init = var_init
        right_var_init = var_init 
    
    if use_ewma_mean:
        
        #return_frame = pair_df.apply(lambda C: ewma_mean_dist(C, lambda_value, mean_initial_value=0))
        return_frame = df
        
        return_frame['Left_EWMA_Mean'] = ewma(return_frame['Left_Returns'].to_frame(), lambda_value, initial_value=0)
        return_frame['Right_EWMA_Mean'] = ewma(return_frame['Right_Returns'].to_frame(), lambda_value, initial_value=0)
        
        return_frame['Covar'] = (return_frame['Left_Returns'] - return_frame['Left_EWMA_Mean']) * (return_frame['Right_Returns'] - return_frame['Right_EWMA_Mean'])
    
        return_frame['EWMA_Covar'] = ewma(return_frame['Covar'].to_frame(),lambda_value, initial_value=covar_init)
    
        return_frame['Left_EWMA_Var'] = ewma_var(return_frame['Left_Returns'].to_frame(), lambda_value, variance_initial_value=left_var_init, mean_initial_value=0)
        return_frame['Right_EWMA_Var'] = ewma_var(return_frame['Right_Returns'].to_frame(), lambda_value, variance_initial_value=right_var_init, mean_initial_value=0)
        
        
    else: # should we allow anything else than zero?
        
        return_frame = df.apply(lambda C: ewma_mean_dist(C, lambda_value, x_bar=np.mean(C)))
        
        return_frame['Covar'] = return_frame.iloc[:,0] * return_frame.iloc[:,1]
    
        return_frame['EWMA_Covar'] = ewma(return_frame['Covar'].to_frame(),lambda_value, initial_value=covar_init)
        
        return_frame['Left_EWMA_Var'] = ewma_var(return_frame['Left_Returns'].to_frame(), lambda_value, variance_initial_value=left_var_init, x_bar=0)
        return_frame['Right_EWMA_Var'] = ewma_var(return_frame['Right_Returns'].to_frame(), lambda_value, variance_initial_value=right_var_init, x_bar=0)
        
    return_frame['EWMA_Correl'] = return_frame['EWMA_Covar'] / (return_frame['Left_EWMA_Var'] * return_frame['Right_EWMA_Var']) ** 0.5
    
    return return_frame

In [41]:
# Confirm "old approach" still works with a single var_init and ewma mean off
ewma_correl_df(pair_in, lambda_value, covar_init=(0.005625 * 0.5), var_init=0.005625, use_ewma_mean=False)
const_mu_correls = ewma_correl_df(pair_in, lambda_value, covar_init=(0.005625 * 0.5), var_init=0.005625, use_ewma_mean=False)
my_assert(0.74933278113088, const_mu_correls['EWMA_Correl'].loc[pd.Timestamp(2018,6,30)])

0.7493327811308803

In [42]:
# Confirm the "new approach" with separate initialisation values 
ewma_correls = ewma_correl_df(pair_in, lambda_value, covar_init=(0.005625 * 0.5), use_ewma_mean=True, left_var_init=0.011957245526038, right_var_init=0.006960204394972)
ewma_correls

,Left_Returns,Right_Returns,Left_EWMA_Mean,Right_EWMA_Mean,Covar,EWMA_Covar,Left_EWMA_Var,Right_EWMA_Var,EWMA_Correl
_valueDateUtc,,,,,,,,,
1960-03-31 00:00:00+00:00,-0.108461,-0.053257,-0.001085,-0.000533,0.005661,0.002841,0.011953,0.006918,0.312413
1960-06-30 00:00:00+00:00,0.020698,-0.030606,-0.000867,-0.000833,-0.000642,0.002806,0.011838,0.006858,0.311437
1960-09-30 00:00:00+00:00,-0.074707,0.040560,-0.001605,-0.000419,-0.002996,0.002748,0.011773,0.006806,0.306999
1960-12-31 00:00:00+00:00,0.071239,-0.040048,-0.000877,-0.000816,-0.002829,0.002692,0.011707,0.006754,0.302785
1961-03-31 00:00:00+00:00,0.106261,0.126850,0.000195,0.000461,0.013406,0.002799,0.011703,0.006846,0.312767
1961-06-30 00:00:00+00:00,-0.010592,-0.043942,0.000087,0.000017,0.000469,0.002776,0.011587,0.006797,0.312836
1961-09-30 00:00:00+00:00,0.024628,-0.088271,0.000332,-0.000866,-0.002124,0.002727,0.011477,0.006805,0.308592
1961-12-31 00:00:00+00:00,0.072056,0.022813,0.001049,-0.000629,0.001665,0.002717,0.011413,0.006743,0.309682
1962-03-31 00:00:00+00:00,-0.036114,-0.029016,0.000678,-0.000913,0.001034,0.002700,0.011312,0.006683,0.310502


In [43]:
# We have a test value now for EWMA correlation, @EndJun2018, it is 0.707718865258392
my_assert(0.707718865258392, ewma_correls['EWMA_Correl'].loc[pd.Timestamp(2018,6,30)])

0.707718865258392

### Over pairs of series without a matching lifespan

We can test that we are successfully truncating the pair of data Series so that we correctly calculate correlation over the shared lifespan.

In [44]:
# take pair and remove data to match spreadsheet example
df = df.apply(pd.to_numeric, errors='ignore')
pair_in = pd.concat([df['E_USD'],df['E_GBP']], axis=1)
pair_in = pair_in[pair_in.index <= pd.Timestamp(2018, 6, 30).tz_localize('GMT')]
pair_in

,E_USD,E_GBP
_valueDateUtc,,
1960-03-31 00:00:00+00:00,-0.108461,-0.053257
1960-06-30 00:00:00+00:00,0.020698,-0.030606
1960-09-30 00:00:00+00:00,-0.074707,0.040560
1960-12-31 00:00:00+00:00,0.071239,-0.040048
1961-03-31 00:00:00+00:00,0.106261,0.126850
1961-06-30 00:00:00+00:00,-0.010592,-0.043942
1961-09-30 00:00:00+00:00,0.024628,-0.088271
1961-12-31 00:00:00+00:00,0.072056,0.022813
1962-03-31 00:00:00+00:00,-0.036114,-0.029016


In [45]:
pair_in.loc[:pd.Timestamp(1962, 3, 31),['E_GBP']] = np.nan
pair_in

,E_USD,E_GBP
_valueDateUtc,,
1960-03-31 00:00:00+00:00,-0.108461,NaN
1960-06-30 00:00:00+00:00,0.020698,NaN
1960-09-30 00:00:00+00:00,-0.074707,NaN
1960-12-31 00:00:00+00:00,0.071239,NaN
1961-03-31 00:00:00+00:00,0.106261,NaN
1961-06-30 00:00:00+00:00,-0.010592,NaN
1961-09-30 00:00:00+00:00,0.024628,NaN
1961-12-31 00:00:00+00:00,0.072056,NaN
1962-03-31 00:00:00+00:00,-0.036114,NaN


In [46]:
ewma_correls = ewma_correl_df(pair_in, lambda_value, covar_init=(0.005625 * 0.5), use_ewma_mean=True, left_var_init=0.011957245526038, right_var_init=0.006960204394972)

# Test value for EWMA correlation with a truncated series, @EndJun2018, it is 0.704303706118467
my_assert(0.704303706118467, ewma_correls['EWMA_Correl'].loc[pd.Timestamp(2018,6,30)])

0.704303706118466

### As a function of pairs of Series

When running the correlations as part of a tool, it is more likely that we will be processing a *Correlation Matrix* based on a set of returns from a number of assets, where we will produce the correlation between each pair of assets. For this approach, a function to apply correlation to individual pairs, rather than a single DataFrame will be useful.  

In [47]:
# take pair and remove data to match spreadsheet example
df = df.apply(pd.to_numeric, errors='ignore')
pair_in = pd.concat([df['E_USD'],df['E_GBP']], axis=1)
pair_in = pair_in[pair_in.index <= pd.Timestamp(2018, 6, 30).tz_localize('GMT')]
pair_in

pd.concat([pair_in.iloc[:,0],pair_in.iloc[:,1]], axis = 1)

,E_USD,E_GBP
_valueDateUtc,,
1960-03-31 00:00:00+00:00,-0.108461,-0.053257
1960-06-30 00:00:00+00:00,0.020698,-0.030606
1960-09-30 00:00:00+00:00,-0.074707,0.040560
1960-12-31 00:00:00+00:00,0.071239,-0.040048
1961-03-31 00:00:00+00:00,0.106261,0.126850
1961-06-30 00:00:00+00:00,-0.010592,-0.043942
1961-09-30 00:00:00+00:00,0.024628,-0.088271
1961-12-31 00:00:00+00:00,0.072056,0.022813
1962-03-31 00:00:00+00:00,-0.036114,-0.029016


In [48]:
def ewma_correl(left_series, right_series, lambda_value, covar_init, var_init=None, left_var_init=None, right_var_init=None, use_ewma_mean=True):
    """ From a Pair of series containing a pair of values, calculate the EWMA correlation between them.
        This is simpler to illustrate through the DataFrame version of the function, so we will call that then read off the correlation series as the result.
    """
    
    pair_df = pd.concat([left_series,right_series], axis = 1).dropna()
    
    correl_df = ewma_correl_df(pair_df, lambda_value, covar_init, var_init=var_init, left_var_init=left_var_init, right_var_init=right_var_init, use_ewma_mean=use_ewma_mean)
        
    return correl_df['EWMA_Correl'].squeeze()

In [49]:
func_correl = ewma_correl(pair_in.iloc[:,0], pair_in.iloc[:,1], lambda_value, covar_init=(0.005625 * 0.5), left_var_init=0.011957245526038, right_var_init=0.006960204394972, use_ewma_mean=True)
my_assert(0.707718865258392, func_correl.loc[pd.Timestamp(2018,6,30)])

0.707718865258392

We can see this being applied to a small set of returns as follows:


In [50]:
url1 = "http://sdi/Api/models/2019-03-31/TimeSeries.MarketData.Equity.ExcessReturn.3m/Best_Views/RW/NONE/NONE/E_GBP/NONE/?newFormat=True"
url2 = "http://sdi/Api/models/2019-03-31/TimeSeries.MarketData.Equity.ExcessReturn.3m/Best_Views/RW/NONE/NONE/E_USD/NONE/?newFormat=True"
url3 = "http://sdi/Api/models/2019-03-31/TimeSeries.MarketData.Equity.ExcessReturn.3m/Best_Views/RW/NONE/NONE/E_AUD/NONE/?newFormat=True"
url4 = "http://sdi/Api/models/2019-03-31/TimeSeries.MarketData.Equity.ExcessReturn.3m/Best_Views/RW/NONE/NONE/E_BRL/NONE/?newFormat=True"

cf = mp.model_dataframe_from_url_list([url1,url2,url3,url4],"_valueDateUtc","EquityAsset")
cf = cf.reset_index()
cf["_valueDateUtc"] = pd.to_datetime(cf["_valueDateUtc"])
cf.set_index(["_valueDateUtc"], inplace = True)
cf.sort_index(inplace=True)
cf = cf.apply(pd.to_numeric, errors='ignore')
cf

EquityAsset,E_AUD,E_BRL,E_GBP,E_USD
_valueDateUtc,,,,
1960-03-31 00:00:00+00:00,0.045675,NaN,-0.053257,-0.108461
1960-06-30 00:00:00+00:00,0.041720,NaN,-0.030606,0.020698
1960-09-30 00:00:00+00:00,0.018553,NaN,0.040560,-0.074707
1960-12-31 00:00:00+00:00,-0.124337,NaN,-0.040048,0.071239
1961-03-31 00:00:00+00:00,0.053712,NaN,0.126850,0.106261
1961-06-30 00:00:00+00:00,0.097758,NaN,-0.043942,-0.010592
1961-09-30 00:00:00+00:00,-0.040702,NaN,-0.088271,0.024628
1961-12-31 00:00:00+00:00,0.010861,NaN,0.022813,0.072056
1962-03-31 00:00:00+00:00,0.036511,NaN,-0.029016,-0.036114


In [51]:
# The "new" approach will require a dictionary of initialisation values
init_dict = {}
init_dict['E_USD'] = 0.011957245526038
init_dict['E_GBP'] = 0.006960204394972
init_dict['E_AUD'] = 0.007747769515651
init_dict['E_BRL'] = 0.031084779327745

In [52]:
headers = list(cf)

from itertools import combinations

#correls = pd.DataFrame([pair for pair in combinations(headers, 2)])
# Confirm "old approach" still works with a single var_init and ewma mean off
ewma_correl_df(pair_in, lambda_value, covar_init=(0.005625 * 0.5), var_init=0.005625, use_ewma_mean=False)

l = []

for pair in combinations(headers, 2):
    # Correlation, using the "new" style
    cor= ewma_correl(cf[pair[0]], cf[pair[1]],
                     lambda_value=0.99, covar_init=(0.005625 * 0.5),
                     left_var_init=init_dict[pair[0]], right_var_init=init_dict[pair[1]],
                     use_ewma_mean=True)
    l.append((pair[0], pair[1], cor.loc[pd.Timestamp(2018,6,30)]))
    
l

[('E_AUD', 'E_BRL', 0.2879237893897236),
 ('E_AUD', 'E_GBP', 0.6178940152907221),
 ('E_AUD', 'E_USD', 0.615246229197829),
 ('E_BRL', 'E_GBP', 0.29052601342233725),
 ('E_BRL', 'E_USD', 0.2787260164485031),
 ('E_GBP', 'E_USD', 0.707718865258392)]

### from MercurPy

For use in later specification documents, and as a check against the future Mercury implementation, we will add the EWMA correlation functions to the *MercurPy* python project so that they can be accessed later.  

In [53]:
# take pair and remove missing
df = df.apply(pd.to_numeric, errors='ignore')
pair_in = pd.concat([df['E_USD'],df['E_GBP']], axis=1).dropna()
pair_in = pair_in[pair_in.index <= pd.Timestamp(2018,6,30).tz_localize('GMT')]
pair_in.columns = ["Left_Returns", "Right_Returns"]
pair_in

,Left_Returns,Right_Returns
_valueDateUtc,,
1960-03-31 00:00:00+00:00,-0.108461,-0.053257
1960-06-30 00:00:00+00:00,0.020698,-0.030606
1960-09-30 00:00:00+00:00,-0.074707,0.040560
1960-12-31 00:00:00+00:00,0.071239,-0.040048
1961-03-31 00:00:00+00:00,0.106261,0.126850
1961-06-30 00:00:00+00:00,-0.010592,-0.043942
1961-09-30 00:00:00+00:00,0.024628,-0.088271
1961-12-31 00:00:00+00:00,0.072056,0.022813
1962-03-31 00:00:00+00:00,-0.036114,-0.029016


In [54]:
my_assert(0.707718865258392, mp.ewma_correl(pair_in.iloc[:,0], pair_in.iloc[:,1], lambda_value, covar_init=(0.005625 * 0.5), left_var_init=0.011957245526038, right_var_init=0.006960204394972, use_ewma_mean=True).loc[pd.Timestamp(2018,6,30)])

0.707718865258392

## Spread Level, Dispersion and Vol

With the comparison workbook open and the "EWMA_LevDispVol_Spread_Current" selected, running the code below will confirm that we can replicate the calculations for Spreads.  


In [55]:
from xlwings import Range

df = Range("B7").options(pd.DataFrame, index=True, header=1,expand='table').value

raw_ewma_mean = ewma(df['Raw Data'], lambda_value = 1-1/(25*12), initial_value = 55.5542857142857)

raw_ewma_var = ewma_var(df['Raw Data'], lambda_value = 1-1/(25*12), variance_initial_value=32.3240599674733**2,mean_initial_value=0)

raw_ewma_vol = ewma_vol(df['Raw Data'], lambda_value = 1-1/(25*12), variance_initial_value=32.3240599674733**2,mean_initial_value=55.5542857142857, steps_per_annum=1)

diff_ewma_vol = ewma_vol(df['Raw Data'].diff(12).dropna(), lambda_value = 1-1/(25*12), variance_initial_value=29.8759823919185**2,mean_initial_value=0, steps_per_annum=1)


print("EWMA Mean result: {0}".format(my_assert(raw_ewma_mean.loc[pd.Timestamp(2018,4,30)],85.8154526607463)))
print("EWMA Dispersion result: {0}".format(my_assert(raw_ewma_vol.loc[pd.Timestamp(2018,4,30)],49.3719413786173)))
print("EWMA Volatility result: {0}".format(my_assert(diff_ewma_vol.loc[pd.Timestamp(2018,4,30)],51.6007990838829)))

KeyError: 'Raw Data'

We are working with larger numbers, so our errors Vs. Excel precision are larger.

## Spread correlation

Similarly we can replicate the correlation calculation on sheet "EWMA_Correl_Spreads_Current".

I need to add an additional date in cell B6 so that the results can be read as a table.

In [ ]:
cf = Range("B5").options(pd.DataFrame, index=True, header=1,expand='table').value

cf=cf.iloc[1:]
cf

In [ ]:
C8A1_ann_diff = cf['C8A1'].diff(12).dropna()
C8A4_ann_diff = cf['C8A4'].diff(12).dropna()

C8A1_C8A4_correl = mp.ewma_correl(C8A1_ann_diff, C8A4_ann_diff, lambda_value=1-1/(25*12), covar_init=1365.41516847966, left_var_init=29.8759823919185**2, right_var_init=53.5767574812328**2, use_ewma_mean=True)

print("EWMA Correl result: {0}".format(my_assert(C8A1_C8A4_correl.loc[pd.Timestamp(2018,4,30)],0.852383156415978)))